## **NBA Transformation**

In [24]:
input_dir = "./xlsx_files"
output_dir = "./csv_files"
debug = True

### Reading the data into a Pandas Dataframe

In [25]:
import os
import pandas as pd


def get_xlsx_files(input_dir):
    xlsx_files = []
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(".xlsx"):
                xlsx_files.append(os.path.join(root, file))
    return xlsx_files

df = pd.DataFrame()
for file in get_xlsx_files(input_dir):
    df = pd.concat([df, pd.read_excel(file)])

if debug: 
    print(df.dtypes)
    print(df.head(5))


Rk          int64
Player     object
Pos        object
Age         int64
Tm         object
G           int64
GS          int64
MP          int64
FG          int64
FGA         int64
FG%       float64
3P          int64
3PA         int64
3P%       float64
2P          int64
2PA         int64
2P%       float64
eFG%      float64
FT          int64
FTA         int64
FT%       float64
ORB         int64
DRB         int64
TRB         int64
AST         int64
STL         int64
BLK         int64
TOV         int64
PF          int64
PTS         int64
Year        int64
dtype: object
   Rk            Player Pos  Age   Tm   G  GS    MP   FG   FGA  ...  ORB  DRB  \
0   1  Precious Achiuwa   C   23  TOR  55  12  1140  196   404  ...  100  228   
1   2      Steven Adams   C   29  MEM  42  42  1133  157   263  ...  214  271   
2   3       Bam Adebayo   C   25  MIA  75  75  2598  602  1114  ...  184  504   
3   4      Ochai Agbaji  SG   22  UTA  59  22  1209  165   386  ...   43   78   
4   5      Santi Aldama

### Changing the column types and selecting the relevant columns

In [26]:
for column in df.columns:
    if debug: print(f'{column}: {df[column].dtype}')
    if df[column].dtype == 'object':
        df = df.astype({column: 'string'})

if debug: print(df.dtypes)

relevant_columns = [
    'Player',
    'Pos',
    'G',
    'MP',
    'TRB',
    'AST',
    'STL',
    'BLK',
    'PTS',
    'Year'
]

for column in df.columns:
    if column not in relevant_columns:
        if debug: print(f'Dropping {column}')
        df = df.drop(column, axis=1)
if debug: print(df.dtypes)

df = df.rename(columns={
    'Pos': 'Position',
    'G': 'Games',
    'MP': 'Minutes',
    'TRB': 'Rebounds',
    'AST': 'Assists',
    'STL': 'Steals',
    'BLK': 'Blocks',
    'PTS': 'Points'
})

if debug: print(df.dtypes)

Rk: int64
Player: object
Pos: object
Age: int64
Tm: object
G: int64
GS: int64
MP: int64
FG: int64
FGA: int64
FG%: float64
3P: int64
3PA: int64
3P%: float64
2P: int64
2PA: int64
2P%: float64
eFG%: float64
FT: int64
FTA: int64
FT%: float64
ORB: int64
DRB: int64
TRB: int64
AST: int64
STL: int64
BLK: int64
TOV: int64
PF: int64
PTS: int64
Year: int64
Rk                 int64
Player    string[python]
Pos       string[python]
Age                int64
Tm        string[python]
G                  int64
GS                 int64
MP                 int64
FG                 int64
FGA                int64
FG%              float64
3P                 int64
3PA                int64
3P%              float64
2P                 int64
2PA                int64
2P%              float64
eFG%             float64
FT                 int64
FTA                int64
FT%              float64
ORB                int64
DRB                int64
TRB                int64
AST                int64
STL                int64
BL

### Change player names to uppercase and filtering players

In [27]:
df['Player'] = df['Player'].map(lambda x: x.upper())

if debug: print(df.head(5))

relevant_players = [
    'LEBRON JAMES',
    'JAMES HARDEN',
    'STEPHEN CURRY',
    'GIANIS ANTETOKOUNMPO'   
]

df = df[df['Player'].isin(relevant_players)]

if debug: print(df.head(5))

             Player Position  Games  Minutes  Rebounds  Assists  Steals  \
0  PRECIOUS ACHIUWA        C     55     1140       328       50      31   
1      STEVEN ADAMS        C     42     1133       485       97      36   
2       BAM ADEBAYO        C     75     2598       688      240      88   
3      OCHAI AGBAJI       SG     59     1209       121       67      16   
4      SANTI ALDAMA       PF     77     1682       371       97      45   

   Blocks  Points  Year  
0      30     508  2023  
1      46     361  2023  
2      61    1529  2023  
3      15     467  2023  
4      48     696  2023  
            Player Position  Games  Minutes  Rebounds  Assists  Steals  \
139  STEPHEN CURRY       PG     56     1941       341      352      52   
245   JAMES HARDEN       PG     58     2135       354      618      71   
306   LEBRON JAMES       PF     55     1954       457      375      50   
135  STEPHEN CURRY       PG     80     2613       341      619     163   
252   JAMES HARDEN     

### Create Dimensional Model

In [28]:
### Defining functions to create the star schema
from typing import Dict, List

def create_dimensions(df : pd.DataFrame, dimensions: Dict[str, List[str]], debug = False):
    if debug : print(dimensions)
    dimensions_dfs = {}
    
    for dimension in dimensions.keys():
        if debug : 
            print(f'Creating dimension: {dimension}')
            print(dimensions[dimension])
        dimension_df = df[dimensions[dimension]].drop_duplicates()
        sk_name = f'sk_{dimension.removeprefix("dim_")}'
        if debug : print(sk_name)
        dimension_df = dimension_df.reset_index(drop=True).reset_index().rename(columns={'index':sk_name})
        dimensions_dfs[dimension] = dimension_df
        if debug : print(dimensions_dfs[dimension])
    
    return dimensions_dfs

def create_facts(df : pd.DataFrame, dimensions_dfs : Dict[str, pd.DataFrame], model : Dict[str, Dict[str, List[str]]], debug = False):
    facts_dfs = {}
    facts = model['facts']
    dimensions = model['dimensions']
    
    if debug :
        print(f'Facts model: {facts}')
        print(f'Original DF head: \n{df.head(5)}')

    for fact in facts.keys():
        facts_dfs[fact] = df
        if debug : 
            print(f'Creating fact: {fact}')
            print(facts[fact])
        for dimension in dimensions_dfs.keys():
            if debug :
                print(f'Merging {fact} with {dimension}')
                #print(facts_dfs[fact].head(5))
                #print(dimensions_dfs[dimension].head(5))
                print(f'Dimension names: {dimensions[dimension]}')
            facts_dfs[fact] = pd.merge(facts_dfs[fact], dimensions_dfs[dimension])
            facts_dfs[fact] = facts_dfs[fact].drop(dimensions[dimension], axis=1)
            if debug : print(facts_dfs[fact].head(5))
    
    return facts_dfs


def create_star(df : pd.DataFrame, model : dict):
    dimensions_dfs = create_dimensions(df, model["dimensions"])
    facts_dfs = create_facts(df, dimensions_dfs, model)

    return facts_dfs, dimensions_dfs

In [29]:
modelo = {
    "facts":{
        "fato_nba" : [
            "Assists",
            "Blocks",
            "Games",
            "Minutes",
            "Points",
            "Rebounds",
            "Steals"
            ]
    },
    "dimensions":{
        "dim_ano" : ["Year"],
        "dim_jogador": ["Player", "Position"],
    }
}

facts_dfs, dimensions_dfs = create_star(df, modelo)

print(f'Head of facts_dfs["fato_nba"]\n{facts_dfs["fato_nba"].head(5)}')
print(f'Head of dimensions_dfs["dim_ano"]\n{dimensions_dfs["dim_ano"].head(5)}')
print(f'Head of dimensions_dfs["dim_jogador"]\n{dimensions_dfs["dim_jogador"].head(5)}')

Head of facts_dfs["fato_nba"]
   Games  Minutes  Rebounds  Assists  Steals  Blocks  Points  sk_ano  \
0     56     1941       341      352      52      20    1648       0   
1     58     2135       354      618      71      31    1216       0   
2     55     1954       457      375      50      32    1590       0   
3     80     2613       341      619     163      16    1900       1   
4     81     2981       459      565     154      60    2217       1   

   sk_jogador  
0           0  
1           1  
2           2  
3           0  
4           3  
Head of dimensions_dfs["dim_ano"]
   sk_ano  Year
0       0  2023
1       1  2015
2       2  2014
3       3  2017
4       4  2021
Head of dimensions_dfs["dim_jogador"]
   sk_jogador         Player Position
0           0  STEPHEN CURRY       PG
1           1   JAMES HARDEN       PG
2           2   LEBRON JAMES       PF
3           3   JAMES HARDEN       SG
4           4   LEBRON JAMES       SF


### Save to csv

In [30]:
import os
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

for fact in facts_dfs.keys():
    facts_dfs[fact].to_csv(os.path.join(output_dir, f'{fact}.csv'), index=False)

for dimension in dimensions_dfs.keys():
    dimensions_dfs[dimension].to_csv(os.path.join(output_dir, f'{dimension}.csv'), index=False)